<a href="https://colab.research.google.com/github/Nilufayeasmin299/Reproduce-GNN_Ownership_Verification/blob/main/Revisiting_Blackbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/rkzhou/GNN_OwnVer.git
%cd GNN_OwnVer


Cloning into 'GNN_OwnVer'...
remote: Enumerating objects: 435, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 435 (delta 13), reused 2 (delta 2), pack-reused 395 (from 1)
Receiving objects: 100% (435/435), 1.10 MiB | 3.75 MiB/s, done.
Resolving deltas: 100% (252/252), done.
/content/GNN_OwnVer


In [ ]:
# Install PyTorch (check your CUDA version from https://pytorch.org/get-started/locally/)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install PyTorch-Geometric dependencies
!pip install torch-scatter torch-sparse torch-geometric -f https://data.pyg.org/whl/torch-2.0.1+cu118.html

# Install additional dependencies (if any are needed)
!pip install scikit-learn pandas matplotlib tqdm


Looking in indexes: https://download.pytorch.org/whl/cu118
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00


In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0))


GPU available: True
Device: Tesla T4


In [ ]:
!ls


config	main  model  README.md	utils


In [ ]:
from torch_geometric.datasets import Planetoid

# Load the Cora dataset
dataset = Planetoid(root='data/', name='Cora')
data = dataset[0]
print(data)


/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
Processing...


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


Done!


In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Load data to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
data = data.to(device)

# Training setup
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 1.9403363466262817
Epoch 2, Loss: 1.8325576782226562
Epoch 3, Loss: 1.6998648643493652
Epoch 4, Loss: 1.5655665397644043
Epoch 5, Loss: 1.4439955949783325
Epoch 6, Loss: 1.3191763162612915
Epoch 7, Loss: 1.1970280408859253
Epoch 8, Loss: 1.081295132637024
Epoch 9, Loss: 0.9693129658699036
Epoch 10, Loss: 0.8604828715324402
Epoch 11, Loss: 0.7556288838386536
Epoch 12, Loss: 0.6567628383636475
Epoch 13, Loss: 0.5651360154151917
Epoch 14, Loss: 0.48133358359336853
Epoch 15, Loss: 0.4061400592327118
Epoch 16, Loss: 0.3400634825229645
Epoch 17, Loss: 0.28329381346702576
Epoch 18, Loss: 0.23557956516742706
Epoch 19, Loss: 0.19605179131031036
Epoch 20, Loss: 0.16354717314243317
Epoch 21, Loss: 0.13692758977413177
Epoch 22, Loss: 0.11513674259185791
Epoch 23, Loss: 0.09725823253393173
Epoch 24, Loss: 0.08255264163017273
Epoch 25, Loss: 0.0703980103135109
Epoch 26, Loss: 0.06027870997786522
Epoch 27, Loss: 0.051813866943120956
Epoch 28, Loss: 0.04470890760421753
Epoch 29, Loss: 0

In [ ]:

def mask_features(data, mask_ratio=0.1):
    # Ensure the mask is created on the same device as data.x
    mask = (torch.rand(data.x.shape, device=data.x.device) > mask_ratio)
    data.x = data.x * mask
    return data

# Apply masking to the dataset
data = mask_features(data)




In [ ]:
from sklearn.neural_network import MLPClassifier

# Example data for verification classifier
features = torch.rand(100, 16).numpy()  # Placeholder features
labels = torch.randint(0, 2, (100,)).numpy()  # Placeholder labels

# Train verification classifier
clf = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu')
clf.fit(features, labels)

print("Verification accuracy:", clf.score(features, labels))


Verification accuracy: 1.0


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
# Save model
torch.save(model.state_dict(), 'gcn_model.pth')

# Load model
model.load_state_dict(torch.load('gcn_model.pth'))


<ipython-input-13-02b96214445d>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('gcn_model.pth'))


<All keys matched successfully>